In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from senet import SENet  # Replace 'senet' with the actual package name or path to the SENet implementation

# Define constants
NUM_CLASSES = 4  # Number of classes (correct, partially correct, incorrect, none)
IMG_WIDTH, IMG_HEIGHT = 224, 224  # Input image dimensions for SENet
BATCH_SIZE = 32
EPOCHS = 15

# Load the SENet model (replace 'SENet' with the actual model name from the third-party implementation)
base_model = SENet(weights='imagenet', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

# Freeze the base model's layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top of the base model
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)  # Adding dropout to prevent overfitting
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)  # Adding another dropout layer
x = tf.keras.layers.Dense(256, activation='relu')(x)  # Adding another Dense layer
x = tf.keras.layers.Dropout(0.5)(x)  # Adding another dropout layer
x = tf.keras.layers.Dense(128, activation='relu')(x)  # Adding another Dense layer
x = tf.keras.layers.Dropout(0.5)(x)  # Adding another dropout layer
x = tf.keras.layers.Dense(64, activation='relu')(x)  # Adding another Dense layer
x = tf.keras.layers.Dropout(0.5)(x)  # Adding another dropout layer
x = tf.keras.layers.Dense(32, activation='relu')(x)  # Adding another Dense layer
x = tf.keras.layers.Dropout(0.5)(x)  # Adding another dropout layer
predictions = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Create data generators for training and validation
TRAINING_DIR = r"C:/Users/Lenovo/Desktop/koi bhi/train/"
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

VALIDATION_DIR = r"C:/Users/Lenovo/Desktop/koi bhi/test/"
validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // BATCH_SIZE
)

# Save the model for later use
model.save('pose_classification_model_new_added_layer_SENet.h5')
